<h3 style="color:#4A90E2;">💡 Intelligent Health-Record Assistant</h3>

<p style="color:#333;">
The <strong>Intelligent Health-Record Assistant</strong> is an AI-powered system built to understand patient queries, classify medical intents, and deliver meaningful responses with the help of structured memory and context tracking.
</p>

<p style="color:#444;">
It keeps recent conversation history, detects intents such as <span style="color:#1E90FF;">symptom descriptions</span>, <span style="color:#1E90FF;">medication clarifications</span>, or <span style="color:#1E90FF;">appointment requests</span>, and responds with clear and supportive guidance.
</p>

<p style="color:#333;">
By providing fast, consistent, and context-aware answers, the assistant helps improve healthcare interactions while keeping each conversation organized and easy to follow.
</p>


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [16]:
pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.

In [17]:
import json
from datetime import datetime 
from dataclasses import dataclass, field
from typing import List, Dict

print("Import successfull")

Import successfull


In [18]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [20]:
import json
import os
import google.generativeai as genai
from datetime import datetime
from dataclasses import dataclass, field
from typing import List, Dict

# ------------------------------------------------------------
# ✔ Google API Key is already set in previous cell via Kaggle Secrets
# ------------------------------------------------------------
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


# 1. Memory
@dataclass
class Memory:
    messages: List[Dict] = field(default_factory=list)
    max_history: int = 20

    def add(self, role, content):
        self.messages.append({
            "role": role,
            "content": content,
            "time": datetime.now().isoformat()
        })
        if len(self.messages) > self.max_history:
            self.messages = self.messages[-self.max_history:]

    def get_context(self):
        out = ""
        for m in self.messages[-5:]:
            out += f"{m['role']}: {m['content']}\n"
        return out


# 2. Intent Classification
class IntentAgent:
    def classify(self, message):
        text = message.lower()

        if "lab" in text or "report" in text:
            return "lab_report_query", "medium"
        if "prescription" in text or "medicine" in text:
            return "prescription_query", "medium"
        if "summarize" in text or "summary" in text:
            return "record_summary", "high"
        if "explain" in text or "what does" in text:
            return "medical_explanation", "medium"
        if "upload" in text or "analyze" in text:
            return "record_upload", "high"
        if "symptom" in text or "feel" in text:
            return "symptom_query", "high"

        return "general_health_query", "low"


# 3. Reply Agent using Gemini
class ReplyAgent:
    def __init__(self):
        self.model = genai.GenerativeModel("gemini-2.5-flash-lite")

    def create_reply(self, message, intent, urgency, context):
        prompt = f"""
You are an Intelligent Health-Record Assistant.

Use the last 5 conversation messages as context.

Context:
{context}

User message:
{message}

Detected intent: {intent}
Urgency: {urgency}

Generate a clear, supportive, medically accurate response.
Avoid giving diagnoses.
"""

        response = self.model.generate_content(prompt)
        return response.text.strip()


# 4. Coordinator
class Coordinator:
    def __init__(self):
        self.intent_agent = IntentAgent()
        self.reply_agent = ReplyAgent()
        self.memory = Memory()

    def ask(self, message):
        self.memory.add("user", message)

        intent, urgency = self.intent_agent.classify(message)
        context = self.memory.get_context()
        reply = self.reply_agent.create_reply(message, intent, urgency, context)

        final = {
            "intent": intent,
            "urgency": urgency,
            "reply": reply
        }

        self.memory.add("agent", reply)
        return final


# Demo
agent = Coordinator()

messages = [
    "Can you summarize this medical record?",
    "My lab report shows high creatinine. Explain it.",
    "I want to understand this prescription.",
    "I am feeling chest pain and dizziness.",
    "I want to upload a report for analysis."
]

for msg in messages:
    print("USER:", msg)
    output = agent.ask(msg)
    print(json.dumps(output, indent=2))
    print("-" * 50)


USER: Can you summarize this medical record?
{
  "intent": "record_summary",
  "urgency": "high",
  "reply": "Please provide me with the medical record you would like me to summarize. I will do my best to provide a clear and helpful overview."
}
--------------------------------------------------
USER: My lab report shows high creatinine. Explain it.
{
  "intent": "lab_report_query",
  "urgency": "medium",
  "reply": "A high creatinine level in your lab report can indicate that your kidneys may not be filtering waste products from your blood as efficiently as they should. Creatinine is a waste product that's produced when you use your muscles, and healthy kidneys filter it out of your blood and excrete it in your urine.\n\nThere are several reasons why creatinine levels might be elevated, including:\n\n*   **Kidney function:** This is the most common reason. If your kidneys are not working well, creatinine can build up in your blood.\n*   **Dehydration:** Not drinking enough fluids can 

In [21]:
print(agent.memory.get_context())

agent: I can help you understand your prescription. Please tell me the name of the medication and any other details you have, such as the dosage or how often you are supposed to take it. I can then explain what the medication is generally used for, how it works, and common side effects to be aware of.
user: I am feeling chest pain and dizziness.
agent: I understand you're experiencing chest pain and dizziness. These symptoms can be concerning and warrant prompt medical attention.

**Please seek immediate medical help.** Call emergency services (like 911 in the US, or your local emergency number) or go to the nearest emergency room right away.

While I am here to help with health information, I cannot provide medical advice or diagnoses. A healthcare professional needs to evaluate your symptoms in person to determine the cause and provide appropriate care.
user: I want to upload a report for analysis.
agent: I can help you with that. Please upload the report, and I'll do my best to assi

In [23]:
agent = Coordinator()

messages = [
    "Please explain my lab report.",
    "What does high bilirubin mean?",
    "I want to upload a prescription."
]

for msg in messages:
    out = agent.ask(msg)
    print(json.dumps(out, indent=2))


{
  "intent": "lab_report_query",
  "urgency": "medium",
  "reply": "I understand you'd like me to explain your lab report. To do that, I need you to provide the lab report information. Please share the details, and I'll do my best to explain them to you in a clear and understandable way."
}
{
  "intent": "medical_explanation",
  "urgency": "medium",
  "reply": "High bilirubin can indicate that your liver is not processing bilirubin as efficiently as it should. Bilirubin is a yellowish pigment that is made during the normal breakdown of red blood cells. When bilirubin levels are high, it can sometimes lead to jaundice, which is a yellowing of the skin and the whites of the eyes.\n\nThere are a few reasons why bilirubin levels might be high, including:\n\n*   **Increased breakdown of red blood cells:** If red blood cells are breaking down faster than usual, your liver can become overwhelmed with bilirubin.\n*   **Liver problems:** Conditions that affect the liver's ability to process bi